In [1]:
import socket

In [3]:
sock=socket.socket()
sock.setblocking(False)
try:
    sock.connect(('xkcd.com',80))
except BlockingIOError:
    pass

In [10]:
#url='http://xkcd.com/'
request='GET {0} HTTP/1.0\r\nHOST: xkcd.com\r\n\r\n'.format(url)
encoded=request.encode('ascii')

while True:
    try:
        sock.send(encoded)
        break
    except OSError as e:
        pass
      
print('sent')

KeyboardInterrupt: 

In [11]:
from selectors import DefaultSelector, EVENT_WRITE

selector = DefaultSelector()

sock = socket.socket()
sock.setblocking(False)
try:
    sock.connect(('xkcd.com', 80))
except BlockingIOError:
    pass

def connected():
    selector.unregister(sock.fileno())
    print('connected!')

selector.register(sock.fileno(), EVENT_WRITE, connected)

SelectorKey(fileobj=1144, fd=1144, events=2, data=<function connected at 0x00930468>)

In [12]:
def loop():
    while True:
        events = selector.select()
        for event_key, event_mask in events:
            callback = event_key.data
            callback()

In [13]:
urls_todo = set(['/'])
seen_urls = set(['/'])

In [20]:
class Fetcher:
    def __init__(self, url):
        self.response = b''  # Empty array of bytes.
        self.url = url
        self.sock = None

    # Method on Fetcher class.
    def fetch(self):
        self.sock = socket.socket()
        self.sock.setblocking(False)
        try:
            self.sock.connect(('xkcd.com', 80))
        except BlockingIOError:
            pass

        # Register next callback.
        selector.register(self.sock.fileno(),
                          EVENT_WRITE,
                          self.connected)
        
            # Method on Fetcher class.
    def connected(self, key, mask):
        print('connected!')
        selector.unregister(key.fd)
        request = 'GET {} HTTP/1.0\r\nHost: xkcd.com\r\n\r\n'.format(self.url)
        self.sock.send(request.encode('ascii'))

        # Register the next callback.
        selector.register(key.fd,
                          EVENT_READ,
                          self.read_response)
        
        # Method on Fetcher class.
    def read_response(self, key, mask):
        global stopped

        chunk = self.sock.recv(4096)  # 4k chunk size.
        if chunk:
            self.response += chunk
        else:
            selector.unregister(key.fd)  # Done reading.
            links = self.parse_links()

            # Python set-logic:
            for link in links.difference(seen_urls):
                urls_todo.add(link)
                Fetcher(link).fetch()  # <- New Fetcher.

            seen_urls.update(links)
            urls_todo.remove(self.url)
            if not urls_todo:
                stopped = True

In [18]:
# Begin fetching http://xkcd.com/353/
fetcher = Fetcher('/353/')
fetcher.fetch()

while True:
    events = selector.select()
    for event_key, event_mask in events:
        callback = event_key.data
        callback(event_key, event_mask)

AttributeError: 'Fetcher' object has no attribute 'connected'

In [22]:
import dis

def foo():
    bar()
def bar():
    pass

In [23]:
dis.dis(bar)

  6           0 LOAD_CONST               0 (None)
              3 RETURN_VALUE


In [24]:
dis.dis(foo)

  4           0 LOAD_GLOBAL              0 (bar)
              3 CALL_FUNCTION            0 (0 positional, 0 keyword pair)
              6 POP_TOP
              7 LOAD_CONST               0 (None)
             10 RETURN_VALUE


In [25]:
def f2(z,a=3,b=4):
    return sum(locals().values())

In [26]:
f2(1)

8

In [29]:
dis.dis(f2)

  2           0 LOAD_GLOBAL              0 (sum)
              3 LOAD_GLOBAL              1 (locals)
              6 CALL_FUNCTION            0 (0 positional, 0 keyword pair)
              9 LOAD_ATTR                2 (values)
             12 CALL_FUNCTION            0 (0 positional, 0 keyword pair)
             15 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
             18 RETURN_VALUE


In [30]:
def gen_fn():
    result=yield 1
    print('result of yield: {}'.format(result))
    result=yield 2
    print('result of 2nd yield: {}'.format(result2))
    return 'done'

In [31]:
generator_bit=1<<5

In [33]:
bool(gen_fn.__code__.co_flags & generator_bit)

True

In [41]:
gen=gen_fn()

In [35]:
type(gen)

generator

In [36]:
gen.gi_code.co_name

'gen_fn'

In [37]:
gen.gi_frame.f_lasti

-1

In [42]:
gen.send(None)

1

In [43]:
gen.gi_frame.f_lasti

3

In [44]:
len(gen.gi_code.co_code)

56

In [45]:
gen.send('hello')

result of yield: hello


2

In [46]:
gen.gi_frame.f_locals

{'result': 'hello'}

In [57]:
def gent():
    yield ['a','b']

In [59]:
g1=gent()
g1.send(None)

['a', 'b']

In [56]:
g1.send(None)

StopIteration: 